In [ ]:
import pandas as pd
import pickle
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding,LSTM,Bidirectional,Dense
from keras.utils import to_categorical
from keras import Sequential
from keras.callbacks import Callback,TensorBoard
import random

In [ ]:
data = pd.read_json('intents.json')

In [ ]:
text = []
label = []
for i in data['intents']:
  for j in i['patterns']:
    text.append(j.lower())
    label.append(i['tag'])

In [ ]:
t_n = Tokenizer(oov_token="<oov>")
t_n.fit_on_texts(text)

In [ ]:
word_ind = len(t_n.word_index) + 1 

In [ ]:
t_n_x = t_n.texts_to_sequences(text)

In [ ]:
max_len = 8
x = pad_sequences(t_n_x,maxlen=max_len,padding='post')

In [ ]:
l_e = LabelEncoder()
y = l_e.fit_transform(label)

In [ ]:
def preprocess(text):
  t_n_x = t_n.texts_to_sequences([text])
  return pad_sequences(t_n_x,maxlen=8,padding='post')
preprocess('hello')

array([[34,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [ ]:
#pickle.dump(t_n, open( "Tokenizer.p", "wb" ) ) #now save Tokenizer

In [ ]:
#pickle.dump( l_e, open( "LabelEncoder.p", "wb" ) ) #save LabelEncoder

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=word_ind,output_dim=len(set(label)),input_length=max_len))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Bidirectional(LSTM(34)))
model.add(Dense(len(set(label)),activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 8, 18)             2124      
_________________________________________________________________
bidirectional (Bidirectional (None, 8, 128)            42496     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 68)                44336     
_________________________________________________________________
dense (Dense)                (None, 18)                1242      
Total params: 90,198
Trainable params: 90,198
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("accuracy") >= 1.0):
          print("accuracy 100%")
          self.model.stop_training = True
ten = TensorBoard("/content/logs",histogram_freq=1)

model.fit(x,to_categorical(y),epochs=1000,callbacks=[CustomCallback(),ten])

Epoch 1/1000
5/5 [==============================] - 36s 246ms/step - loss: 2.8901 - accuracy: 0.0565
Epoch 2/1000
5/5 [==============================] - 0s 9ms/step - loss: 2.8786 - accuracy: 0.1338
Epoch 3/1000
5/5 [==============================] - 0s 10ms/step - loss: 2.8667 - accuracy: 0.1386
Epoch 4/1000
5/5 [==============================] - 0s 7ms/step - loss: 2.8579 - accuracy: 0.1217
Epoch 5/1000
5/5 [==============================] - 0s 8ms/step - loss: 2.8257 - accuracy: 0.1512
Epoch 6/1000
5/5 [==============================] - 0s 8ms/step - loss: 2.8166 - accuracy: 0.1088
Epoch 7/1000
5/5 [==============================] - 0s 8ms/step - loss: 2.7709 - accuracy: 0.2259
Epoch 8/1000
5/5 [==============================] - 0s 9ms/step - loss: 2.7499 - accuracy: 0.1480
Epoch 9/1000
5/5 [==============================] - 0s 8ms/step - loss: 2.7340 - accuracy: 0.1595
Epoch 10/1000
5/5 [==============================] - 0s 9ms/step - loss: 2.6655 - accuracy: 0.1748
Epoch 11/1000
5

In [ ]:
#model.save('chatbot_model.h5') #save main model

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/logs

In [ ]:
#now text
while 1:
  pri = model.predict(preprocess(input("Me ==> ").lower()))
  if pri[0][np.argmax(pri)]*100 >= 65.00:
    for i in data['intents']:
      if i['tag'] == l_e.inverse_transform([np.argmax(pri)])[0]:
        print(f"Bot ==> {random.choice(i['responses'])}")
        break
  else:print("Bot ==> Sorry, Don't able to response this message!")



Me ==> hi
Bot ==> fine
Me ==> hlw
Bot ==> fine
Me ==> hi
Bot ==> Good to see you again!
Me ==> how are you
Bot ==> Sorry, Don't able to response this message!
Me ==> Good bye
Bot ==> Please do not swear
Me ==> good
Bot ==> Goodbye!
Me ==> goodbye
Bot ==> Talk to you later
Me ==> ok
Bot ==> No problem!
Me ==> hy
Bot ==> No problem!
